In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score
from matplotlib import style

In [0]:
class SVM:
    def __init__(self, plot_graph=True):
        self.colors = {1: 'r', -1: 'b'}


    # train
    def fit(self, training_data):
        self.train_data = training_data
        # { ||w||: [w,b] }
        optimum_values = {}
        # Assuming all the possible transforms for the 2d data (To find which quadrant the data lies)
        transforms = [[1, 1],
                      [-1,1],
                      [-1,-1],
                      [1,-1]]

        feature_set = []
        for yi in self.train_data:

            for featureset in self.train_data[yi]:

                for feature in featureset:
                    feature_set.append(feature)

        self.max_feature_value = max(feature_set)
        self.min_feature_value = min(feature_set)
        feature_set = None

        #Asssuming step sizes based on Max Feature value;
        step_sizes = [self.max_feature_value * 0.1,
                      self.max_feature_value * 0.01,]

        b_range_multiple = 1
        b_multiple = 3
        w_optimum = self.max_feature_value * 10

        for step in step_sizes:
            w = np.array([w_optimum, w_optimum])

            optimized = False
            while not optimized:
                #Iterating for all combinations of b
                for b in np.arange(-1 * (self.max_feature_value * b_range_multiple),
                                   self.max_feature_value * b_range_multiple,
                                   step * b_multiple):

                    for transformation in transforms:
                        # Multiply with assumed value of w with each transform
                        w_t = w * transformation
                        found_option = True

                        for i in self.train_data:
                            for xi in self.train_data[i]:
                                yi = i

                                if yi * (np.dot(w_t, xi) + b) < 1:
                                    # If any feature value's dot product is less than 1, then ignore that weight matrix
                                    found_option = False
                                break

                        if found_option:
                            optimum_values[np.linalg.norm(w_t)] = [w_t, b]
                        break

                if w[0] < 0:
                    optimized = True
                    print('Optimized a step.')
                else:
                    w = w - step

            norms = sorted([n for n in optimum_values])
            # ||w|| : [w,b]
            optimum = optimum_values[norms[0]]

            self.w = optimum[0]
            self.b = optimum[1]

            w_optimum = optimum[0][0] + step * 2


    def predict(self, testing_data):
        # sign( x.w+b )
        results = np.sign(np.dot(np.array(testing_data), self.w) + self.b)  #sign(sigma(i to n)[alpha(i)y(i)x(i).x(q)]+b)
        return results

   

In [2]:
%cd /content/drive/My Drive/NLP 2020/Project/Classifiers

/content/drive/My Drive/NLP 2020/Project/Classifiers


In [0]:
header = ['Labels','Feature_0','Product_Name','Feature_1', 'Feature_2']
data_read = pd.read_csv(r"train_dataset.csv",sep=',', usecols=header)
data_read1=data_read[['Labels','Feature_0','Product_Name','Feature_1', 'Feature_2']]


my_dict={}
list_tp=[]
list_tn=[]

In the dataset,feature0 is the review,feature1 is the sentiment value associated with the review and feature2 is the rating

In [0]:
data_read.head()

,Labels,Feature_0,Product_Name,Feature_1,Feature_2
0,1.0,system breeze set amazed easy use everything q...,Echo WhiteEcho White,0.216667,5.0
1,1.0,gift senior citizen fan computers technology m...,Fire Tablet Display WiFi GB Includes Specia...,0.461905,5.0
2,1.0,love voice activated device great sound single...,Echo WhiteEcho White,0.407143,5.0
3,1.0,nice tablet beginner casual user basic functions,Fire Tablet Display WiFi GB Includes Specia...,0.033333,4.0
4,1.0,amazon echo great device assist home automatio...,Amazon Fire Hd Tablet WiFi Gb Special Offers...,0.800000,5.0


In [0]:
for index,row in data_read1.iterrows():
    if row['Labels']==1:
        list_tp.append(row[3:].tolist())
    else:
        list_tn.append(row[3:].tolist())

my_dict[1]=np.asarray(list_tp)
my_dict[-1]=np.asarray(list_tn)


In [12]:
svm = SVM()
svm.fit(training_data=my_dict)

Optimized a step.
Optimized a step.


In [0]:

header = ['Labels','Feature_0','Product_Name','Feature_1', 'Feature_2']
data_read2 = pd.read_csv(r"test_dataset.csv",sep=',', usecols=header)
data_read3=data_read2[['Labels','Feature_0','Product_Name','Feature_1', 'Feature_2']]

list_test_features = []
label = data_read3['Labels']
list_test_labels = []

for index,row in data_read3.iterrows():
        list_test_features.append(row[3:].tolist())
        #print(row[3:].tolist()) feature_1 and feature_2

for x in label:
        #print(x) the label -1 or 1
        list_test_labels.append(x)

predict_us = list_test_features
pred = []

for p in predict_us:
    pred.append(svm.predict(p))



In [14]:
results = confusion_matrix(list_test_labels, pred, labels=[1, -1])
print('Confusion Matrix:')
print(results)

print(classification_report(list_test_labels, pred))
print('Accuracy Score :', accuracy_score(list_test_labels, pred))


Confusion Matrix:
[[5188   55]
 [  78  161]]
              precision    recall  f1-score   support

        -1.0       0.75      0.67      0.71       239
         1.0       0.99      0.99      0.99      5243

    accuracy                           0.98      5482
   macro avg       0.87      0.83      0.85      5482
weighted avg       0.97      0.98      0.98      5482

Accuracy Score : 0.9757387814666181


In [0]:
products_list = data_read3['Product_Name']
products = []
for x in products_list:
        products.append(x)

In [0]:
complete_list = []
def perf_measure(products,y_actual, y_pred):

    for i in range(len(y_pred)):
        TP = 0
        FP = 0
        TN = 0
        FN = 0
        if y_actual[i]==y_pred[i]==1:
           TP += 1
        if y_pred[i]==1 and y_actual[i]!=y_pred[i]:
           FP += 1
        if y_actual[i]==y_pred[i]==-1:
           TN += 1
        if y_pred[i]==-1 and y_actual[i]!=y_pred[i]:
           FN += 1

        complete_list.append([products[i],y_actual[i],y_pred[i],TP, FP, TN, FN])

In [0]:
perf_measure(products, list_test_labels, pred)
header = ['Products_Name','Actual','Predicted','TP','FP','TN','FN']
dfObj = pd.DataFrame(complete_list,columns=header)
dfObj.to_csv("Output_SVM.csv",index=False)

In [0]:
dfObj.head(10)

,Products_Name,Actual,Predicted,TP,FP,TN,FN
0,Fire Tablet Display WiFi GB Includes Specia...,1.0,1.0,1,0,0,0
1,Amazon Kindle Paperwhite eBook reader GB ...,1.0,1.0,1,0,0,0
2,Fire Tablet Display WiFi GB Includes Specia...,1.0,1.0,1,0,0,0
3,Fire Tablet Display WiFi GB Includes Specia...,1.0,1.0,1,0,0,0
4,Amazon Kindle Paperwhite eBook reader GB ...,1.0,1.0,1,0,0,0
5,AllNew Fire HD Tablet HD Display WiFi GB I...,1.0,1.0,1,0,0,0
6,Fire Tablet Display WiFi GB Includes Specia...,1.0,1.0,1,0,0,0
7,Echo WhiteEcho White,1.0,1.0,1,0,0,0
8,Fire Tablet Display WiFi GB Includes Specia...,-1.0,-1.0,0,0,1,0
9,Amazon Fire TvAmazon Fire Tv,1.0,1.0,1,0,0,0


In this notebook,the svm algorithm is implemented.The confusion matrix along with the accuracy,precision,recall and f1 score is printed as the performance measure for the given data.

First the training data is plotted in the n- dimensional space.Then a margin is calculated such that is best seperates the points and the classes in the n-dimensional space,that is ,in the training part,the aim is to obtain the margin such that the distance between the points falling along the edge of the margin is  large.(Maximum magin classifier).Then ,for the test points,they are plotted and depending on which side of the margin the points fall under,the label is determined.

The ouput of each predicted product is saved into a separate csv file "Output_SVM.csv"